In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
p=0.8
q=1-p
a=p
q5=[]
q10=[]
q100=[]
q1000=[]
q100000=[]
for n in [5,10,100,1000,100000]:
    for j in range(5):
        X=[]
        for i in range(n):
            p=-a/np.log(q)
            r=np.random.uniform()
            x=1
            r=r-p
            while r>0:
                x+=1
                p=p*a*(x-1)/x
                r=r-p
            X.append(x)   
        if n < 11:
            print(X)
        if n==5:
            q5.append(X)
        elif n==10:
            q10.append(X)
        elif n==100:
            q100.append(X)
        elif n==1000:
            q1000.append(X)
        elif n==100000:
            q100000.append(X)

[2, 1, 13, 2, 1]
[1, 1, 1, 2, 2]
[1, 3, 1, 1, 2]
[1, 1, 3, 2, 1]
[2, 1, 3, 3, 3]
[1, 1, 1, 6, 1, 9, 4, 1, 2, 1]
[13, 1, 6, 1, 2, 3, 4, 1, 1, 2]
[1, 3, 1, 1, 4, 5, 1, 1, 2, 1]
[4, 4, 1, 2, 3, 1, 1, 4, 2, 2]
[2, 8, 1, 2, 1, 1, 1, 1, 1, 1]


In [3]:
q5=np.asarray(q5)
q10=np.asarray(q10)
q100=np.asarray(q100)
q1000=np.asarray(q1000)
q100000=np.asarray(q100000)

In [4]:
from scipy import stats
def find_nearest(array, value):
    array = np.asarray(array)
    return (np.abs(array - value)).argmin()
     
def sturges(array):
    #https://numpy.org/devdocs/reference/generated/numpy.histogram_bin_edges.html
    values=np.histogram_bin_edges(array, bins='sturges')
    counted=np.zeros(values.shape[0])
    arr,counts=np.unique(array,return_counts=True)
    if array.max() < len(values):
        arr_i=0
        itog=[]
        for i in range(1,arr.max()+1):
            if arr[arr_i]==i:
                itog.append(counts[arr_i])
                arr_i+=1
            else:
                itog.append(0)
        itog=np.asarray(itog)
        return np.array(range(1,arr.max()+1)), itog
    for i,j in enumerate(arr):
        indd=find_nearest(values,j)
        counted[indd]+=counts[i]
    return values,counted
def theory_sturges(array,true_dist):
    values=np.histogram_bin_edges(array, bins='sturges')
    dist=[]
    counted=np.zeros(values.shape[0])
    for i in range(1,array.max()+1):
        dist.append(true_dist.pmf(k=i))
    dist=np.asarray(dist)
    if array.max() < len(values):
        return np.array(range(1, len(dist) + 1)), dist * len(array)
    for i,j in enumerate(dist):
        indd=find_nearest(values,i+1)
        counted[indd]+=j
    return values,counted*len(array)

In [5]:
from scipy import stats
true_dist=stats.logser(0.8)
for q in [q100,q1000,q100000]:
    q.sort()
    print('Выборки размера',len(q[0]))
    nomer=0
    df=len(q[0])-1#(total rows - 1) x (total columns - 1)
    crit1 = stats.chi2.ppf(q=0.95, df=df)
    print("Критический уровень для 0,05: {:.4f}".format(crit1))
    for w in q:
        nomer+=1
        print('Выборка №',nomer,end=',')
        values,freq_gen=sturges(w)
        values,freq_theory=theory_sturges(w,true_dist)
        chi_hand=(((freq_gen-freq_theory)**2)/freq_theory).sum().sum()
        print('Значение критерия {:.4f}'.format(chi_hand))
        if chi_hand<crit1:
            print('Гипотеза 0 не отклоняется')
        else:
            print("Гипотеза отклоняется")

Выборки размера 100
Критический уровень для 0,05: 123.2252
Выборка № 1,Значение критерия 6.1243
Гипотеза 0 не отклоняется
Выборка № 2,Значение критерия 10.4119
Гипотеза 0 не отклоняется
Выборка № 3,Значение критерия 35.1729
Гипотеза 0 не отклоняется
Выборка № 4,Значение критерия 10.6639
Гипотеза 0 не отклоняется
Выборка № 5,Значение критерия 6.8062
Гипотеза 0 не отклоняется
Выборки размера 1000
Критический уровень для 0,05: 1073.6427
Выборка № 1,Значение критерия 19.0023
Гипотеза 0 не отклоняется
Выборка № 2,Значение критерия 21.4063
Гипотеза 0 не отклоняется
Выборка № 3,Значение критерия 13.4876
Гипотеза 0 не отклоняется
Выборка № 4,Значение критерия 11.7026
Гипотеза 0 не отклоняется
Выборка № 5,Значение критерия 7.5259
Гипотеза 0 не отклоняется
Выборки размера 100000
Критический уровень для 0,05: 100735.7325
Выборка № 1,Значение критерия 69.3156
Гипотеза 0 не отклоняется
Выборка № 2,Значение критерия 25.5803
Гипотеза 0 не отклоняется
Выборка № 3,Значение критерия 23.1914
Гипотеза 0 н

In [6]:
from scipy import stats
true_dist=stats.logser(0.4)
for q in [q100,q1000,q100000]:
    q.sort()
    print('Выборки размера',len(q[0]))
    nomer=0
    df=len(q[0])-1#(total rows - 1) x (total columns - 1)
    crit1 = stats.chi2.ppf(q=0.95, df=df)
    print("Критический уровень для 0,05: {:.4f}".format(crit1))
    for w in q:
        nomer+=1
        print('Выборка №',nomer,end=',')
        values1,freq_gen=sturges(w)
        values2,freq_theory=theory_sturges(w,true_dist)
        chi_hand=(((freq_gen-freq_theory)**2)/freq_theory).sum().sum()
        print('Значение критерия {:.4f}'.format(chi_hand))
        if chi_hand<crit1:
            print('Гипотеза 0 не отклоняется')
        else:
            print("Гипотеза отклоняется")

Выборки размера 100
Критический уровень для 0,05: 123.2252
Выборка № 1,Значение критерия 751.2020
Гипотеза отклоняется
Выборка № 2,Значение критерия 378324.7526
Гипотеза отклоняется
Выборка № 3,Значение критерия 46223767.2583
Гипотеза отклоняется
Выборка № 4,Значение критерия 12013.0420
Гипотеза отклоняется
Выборка № 5,Значение критерия 7066.2014
Гипотеза отклоняется
Выборки размера 1000
Критический уровень для 0,05: 1073.6427
Выборка № 1,Значение критерия 552662249.8131
Гипотеза отклоняется
Выборка № 2,Значение критерия 48641762.8597
Гипотеза отклоняется
Выборка № 3,Значение критерия 558827.4063
Гипотеза отклоняется
Выборка № 4,Значение критерия 1869200.2141
Гипотеза отклоняется
Выборка № 5,Значение критерия 1055874.2499
Гипотеза отклоняется
Выборки размера 100000
Критический уровень для 0,05: 100735.7325
Выборка № 1,Значение критерия 239632320392486624.0000
Гипотеза отклоняется
Выборка № 2,Значение критерия 8233245697817.5557
Гипотеза отклоняется
Выборка № 3,Значение критерия 1541626

In [9]:
from scipy import stats
true_dist=stats.logser(0.8)
for q in [q100,q1000,q100000]:
    q.sort()
    print('Выборки размера',len(q[0]))
    nomer=0
    df=len(q[0])-1#(total rows - 1) x (total columns - 1)
    crit1 = stats.chi2.ppf(q=0.95, df=df)
    print("Критический уровень для 0,05: {:.4f}".format(crit1))
    for w in q:
        nomer+=1
        print('Выборка №',nomer,end=',')
        arr,counts=np.unique(w,return_counts=True)
        arr_i=0
        itog=[]
        for i in range(1,arr.max()+1):
            if arr[arr_i]==i:
                itog.append(counts[arr_i])
                arr_i+=1
            else:
                itog.append(0)
        itog=np.asarray(itog)
        dist=[]
        for i in range(1,arr.max()+1):
            dist.append(true_dist.pmf(k=i))
        dist=np.asarray(dist) * len(w)
        chi_hand=(((itog-dist)**2)/dist).sum().sum()
        print('Значение критерия {:.4f}'.format(chi_hand))
        if chi_hand<crit1:
            print('Гипотеза 0 не отклоняется')
        else:
            print("Гипотеза отклоняется")

Выборки размера 100
Критический уровень для 0,05: 123.2252
Выборка № 1,Значение критерия 6.1243
Гипотеза 0 не отклоняется
Выборка № 2,Значение критерия 25.5827
Гипотеза 0 не отклоняется
Выборка № 3,Значение критерия 82.5441
Гипотеза 0 не отклоняется
Выборка № 4,Значение критерия 13.6237
Гипотеза 0 не отклоняется
Выборка № 5,Значение критерия 8.8051
Гипотеза 0 не отклоняется
Выборки размера 1000
Критический уровень для 0,05: 1073.6427
Выборка № 1,Значение критерия 35.6839
Гипотеза 0 не отклоняется
Выборка № 2,Значение критерия 28.6241
Гипотеза 0 не отклоняется
Выборка № 3,Значение критерия 18.2926
Гипотеза 0 не отклоняется
Выборка № 4,Значение критерия 13.6845
Гипотеза 0 не отклоняется
Выборка № 5,Значение критерия 18.4187
Гипотеза 0 не отклоняется
Выборки размера 100000
Критический уровень для 0,05: 100735.7325
Выборка № 1,Значение критерия 181.3794
Гипотеза 0 не отклоняется
Выборка № 2,Значение критерия 51.3215
Гипотеза 0 не отклоняется
Выборка № 3,Значение критерия 63.6547
Гипотеза 0

In [10]:
def g(t,x):
    return x.sum()/t + x.shape[0]/((1 - t) * np.log(1 - t))
def g_derivative(t,x):
    return -x.sum()/(t**2) + len(x)/(((1 - t)**2) * (np.log(1 - t))**2) + len(x)/(((1 - t)**2) * np.log(1 - t))
def newton_rapson_logarithmic(t,q):
    return t-g(t,q)/g_derivative(t,q)

In [11]:
from scipy import stats
for q,size in zip([q10,q100,q1000,q100000],['10','100','1000','100000']):
    q.sort()
    print('Выборки размера',size)
    nomer=0
    df=len(q[0])-1#(total rows - 1) x (total columns - 1)
    crit1 = stats.chi2.ppf(q=0.95, df=df)
    print("Критический уровень для 0,05: {:.4f}".format(crit1))
    for vuborka,nomer in zip(q,[0,1,2,3,4]):
        theta=0.5
        for i in range(10):
            theta=newton_rapson_logarithmic(theta,vuborka)
        true_dist=stats.logser(theta)
        print('Выборка №'+str(nomer+1)+', Оценка параметра:{:.4f}'.format(theta),end=',')
        arr,counts=np.unique(vuborka,return_counts=True)
        arr_i=0
        itog=[]
        for i in range(1,arr.max()+1):
            if arr[arr_i]==i:
                itog.append(counts[arr_i])
                arr_i+=1
            else:
                itog.append(0)
        itog=np.asarray(itog)
        itog=itog/len(q[0])
        dist=[]
        for i in range(1,arr.max()+1):
            dist.append(true_dist.pmf(k=i))
        dist=np.asarray(dist)
        chi_hand=(((itog-dist)**2)/dist).sum().sum()
        print('Значение критерия {:.4f}'.format(chi_hand))
        if chi_hand<crit1:
            print('Гипотеза 0 не отклоняется')
        else:
            print("Гипотеза отклоняется")

Выборки размера 10
Критический уровень для 0,05: 16.9190
Выборка №1, Оценка параметра:0.8245,Значение критерия 1.1479
Гипотеза 0 не отклоняется
Выборка №2, Оценка параметра:0.8769,Значение критерия 1.5822
Гипотеза 0 не отклоняется
Выборка №3, Оценка параметра:0.7153,Значение критерия 0.2637
Гипотеза 0 не отклоняется
Выборка №4, Оценка параметра:0.7885,Значение критерия 1.0439
Гипотеза 0 не отклоняется
Выборка №5, Оценка параметра:0.6899,Значение критерия 1.8450
Гипотеза 0 не отклоняется
Выборки размера 100
Критический уровень для 0,05: 123.2252
Выборка №1, Оценка параметра:0.7619,Значение критерия 0.0860
Гипотеза 0 не отклоняется
Выборка №2, Оценка параметра:0.8182,Значение критерия 0.1959
Гипотеза 0 не отклоняется
Выборка №3, Оценка параметра:0.8637,Значение критерия 0.2225
Гипотеза 0 не отклоняется
Выборка №4, Оценка параметра:0.8056,Значение критерия 0.1293
Гипотеза 0 не отклоняется
Выборка №5, Оценка параметра:0.7927,Значение критерия 0.0961
Гипотеза 0 не отклоняется
Выборки размер